In [1]:
#math
import numpy as np
#utils
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
#neural
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
print(tf.__version__)

2.9.2


In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1layer
session = tf.compat.v1.Session(config=config)

2022-10-20 15:45:03.718972: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-20 15:45:04.410789: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 199 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
!rm -rf jff
!mkdir jff

In [6]:
bw = 14
maxnum = 1<<bw
maxnum

16384

In [7]:
inp_arr = np.array([], dtype=np.int16)
out_arr = np.arange(maxnum, dtype=np.int32)

In [8]:
for i in np.arange(1<<14):
    inp_arr = np.append(inp_arr, np.array([list(np.binary_repr(i, width=14))], dtype=np.int16))

In [9]:
inp_arr.shape

(229376,)

In [10]:
inp_arr = inp_arr.reshape(maxnum, bw)

In [11]:
out_arr = out_arr.reshape(maxnum, 1)

In [12]:
inp_arr.shape

(16384, 14)

In [13]:
out_arr.shape

(16384, 1)

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((inp_arr, out_arr))

2022-10-20 01:00:23.241233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 799 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [39]:
input = keras.Input(shape=(bw))

In [50]:
x = tf.keras.layers.Reshape((1, bw))(input)
x = tf.keras.layers.Conv1D(1, 1, activation='relu', name=f'conv1d')(x)

In [44]:
x.node.

TensorShape([None, 1, 1])

In [45]:
x = tf.keras.layers.Flatten(name='Flatten')(x)

In [46]:
x = tf.keras.layers.Rescaling(out_arr.shape[0], name='normalize')(x)

In [47]:
model = keras.Model(input, x)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 14)]              0         
                                                                 
 reshape_7 (Reshape)         (None, 1, 14)             0         
                                                                 
 conv1d (Conv1D)             (None, 1, 1)              15        
                                                                 
 Flatten (Flatten)           (None, 1)                 0         
                                                                 
 normalize (Rescaling)       (None, 1)                 0         
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.get_layer('conv1d').set_weights([np.zeros(14)])

ValueError: You called `set_weights(weights)` on layer "conv1d" with a weight list of length 1, but the layer was expecting 2 weights. Provided weights: [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0....

In [22]:
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['binary_accuracy', 'mean_squared_error', 'MeanAbsolutePercentageError'])

i=1

In [23]:
loss = 1
while loss >= 0.001:
    clear_output(wait=True)
    fitHistory = model.fit(dataset.shuffle(100).batch(out_arr.shape[0]), epochs=10)
    out = model.predict(inp_arr[:1000])
    loss = fitHistory.history['loss'][0]
    plt.plot(out_arr[:1000],out)
    plt.title(f'loss: {loss} epoch: {i*10}')
    plt.savefig(f'jff/{i}.png')
    plt.close()
    i+=1

Epoch 1/10
1/1 [==============================] - 0s 53ms/step - loss: 12561367.0000 - binary_accuracy: 6.1035e-05 - mean_squared_error: 12561367.0000 - mean_absolute_percentage_error: 134773376.0000
Epoch 2/10
1/1 [==============================] - 0s 43ms/step - loss: 12442420.0000 - binary_accuracy: 6.1035e-05 - mean_squared_error: 12442420.0000 - mean_absolute_percentage_error: 134046528.0000
Epoch 3/10
1/1 [==============================] - 0s 44ms/step - loss: 12324793.0000 - binary_accuracy: 6.1035e-05 - mean_squared_error: 12324793.0000 - mean_absolute_percentage_error: 133323352.0000
Epoch 4/10



KeyboardInterrupt



In [ ]:
out = model.predict(inp_arr)

In [ ]:
!ffmpeg -r 5 -i 'jff/%01d.png' -start_number 2 -i 'jff/%01d.png' -c:v libx264 -r 30 -pix_fmt yuv420p 'jff/output.mp4'